<a href="https://colab.research.google.com/github/129Ashish/Handwritten_DIGIT_Classification/blob/main/Handwritten_DIGIT_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HANDWRITTEN DIGIT CLASSIFICATION

## MNIST/CIFAR

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mlt
import seaborn as sns

In [17]:
import torch
from torch.utils.data import Dataset , DataLoader
import torch.nn as nn
import torch.optim as optim

In [1]:
import tensorflow as tf

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Display the shapes of the loaded data
print("Shape of x_train:", x_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_test:", y_test.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Shape of x_train: (60000, 28, 28)
Shape of y_train: (60000,)
Shape of x_test: (10000, 28, 28)
Shape of y_test: (10000,)


In [10]:
import tensorflow as tf
import torch

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

print("Shape of x_train_tensor:", X_train_tensor.shape)
print("Shape of y_train_tensor:", y_train_tensor.shape)
print("Shape of x_test_tensor:", X_test_tensor.shape)
print("Shape of y_test_tensor:", y_test_tensor.shape)

Shape of x_train_tensor: torch.Size([60000, 28, 28])
Shape of y_train_tensor: torch.Size([60000])
Shape of x_test_tensor: torch.Size([10000, 28, 28])
Shape of y_test_tensor: torch.Size([10000])


In [1]:
import torch
#check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device : {device}")

Using device : cuda


# DATASET CLASS

In [2]:
from torch.utils.data import Dataset,DataLoader

class CustomDataset(Dataset):

  def __init__(self,features,labels):
    self.features=features
    self.labels=labels


  def __len__(self):
    return len(self.features)

  def __getitem__(self,idx):
    return self.features[idx],self.labels[idx]

In [8]:
train_dataset = CustomDataset(X_train_tensor,y_train_tensor)
test_dataset = CustomDataset(X_test_tensor,y_test_tensor)

# DATALOADER CLASS

In [13]:
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=True)

print(f"length of train batch : {len(train_loader)}")
print(f"length of test batch : {len(test_loader)}")

length of train batch : 1875
length of test batch : 313


# DEFINING A MODEL

In [27]:
# defining class for model
class MyNN(nn.Module):

  def __init__(self,input_dim,output_dim,num_hidden_layer,neuron_per_layer,dropout_rate):
    super().__init__()
    #architecture of our neural network
    layer=[]#dynamic logic
    for i in range(num_hidden_layer):
      layer.append(nn.Linear(input_dim,neuron_per_layer))
      layer.append(nn.BatchNorm1d(neuron_per_layer))
      layer.append(nn.ReLU())
      layer.append(nn.Dropout(dropout_rate))
      input_dim =neuron_per_layer#input of second layer will be output of previous layer

    layer.append(nn.Linear(neuron_per_layer,output_dim))

    self.model=nn.Sequential(*layer) # *this symbol unpacks the list and make the layers individually accessed


  def forward(self,x):
    return self.model(x)

In [24]:
learning_rate=0.01
epochs = 100

In [25]:
model = MyNN(1)

model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=learning_rate,weight_decay =1e-4)

# OBJECTIVE FUNCTION

In [28]:
# objective function

def objective(trial):

  #next hyperparameter values from the search space
  num_hidden_layer = trial.suggest_int('num_hidden_layer',1,5)# search space is from 1 to 5
  neuron_per_layer=trial.suggest_int('neuron_per_layer',8,128,step=8)#search space like 8-16-24-32..
  epochs=trial.suggest_int('epochs',10,50,step=10)
  learning_rate=trial.suggest_float('learning_rate',1e-5,1e-1,log=True)#logarithmically selection of learning_rate
  dropout_rate=trial.suggest_float('dropout_rate',0.1,0.5,step=0.1)
  #for implementing dropout we have to add new parameter in the MyNN class to use from here
  batch_size=trial.suggest_categorical('batch_size',[16,32,64,128])#select from these list items
  optimizer_name=trial.suggest_categorical('optimizer',['Adam','SGD','RMSprop'])#select from one of these  but we have to implement if-else statement for this.
  weight_decay=trial.suggest_float('weight_decay',1e-5,1e-3,log=True)#logarithmic selection

  #now the train_loader and test_loader will also be used inside the objective function because it can't be defined previously now.
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader =  DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  #model init
  input_dim = 784
  output_dim = 10

  model = MyNN(input_dim,output_dim,num_hidden_layer,neuron_per_layer,dropout_rate)
  model.to(device)#directed to GPU

  #param init
  # learning_rate=0.01
  # epochs=50


  #optimizer selection
  criterion=nn.CrossEntropyLoss()

  if optimizer_name=='Adam':
    optimimzer=optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay)
  elif optimizer_name=='SGD':
    optimimzer=optim.SGD(model.parameters(),lr=learning_rate,weight_decay=weight_decay)
  else :
    optimimzer=optim.RMSprop(model.parameters(),lr=learning_rate,weight_decay=weight_decay)

  #training loop
  total_epoch_loss = 0.0 # Initialize total_epoch_loss before the loop
  for epoch in range(epochs):
      for batch_features,batch_labels in train_loader:
            # Flatten the image data
            batch_features = batch_features.view(-1, 28 * 28).to(device) # Move data to device
            batch_labels = batch_labels.to(device) # Move labels to device

            #forward pass
            outputs=model(batch_features)
            #loss calculate
            loss=criterion(outputs,batch_labels)
            #gradient zero
            optimimzer.zero_grad() # Corrected optimizer name
            #back propogation
            loss.backward()
            #upgrade gradients
            optimimzer.step() # Corrected optimizer name

    #         total_epoch_loss += loss.item() # Accumulate loss

    # avg_loss=total_epoch_loss/len(train_loader) #avg epoch loss
    # print(f"epoch : {epoch+1}, Loss : {avg_loss}")

  #evaluation
  #set model to eval mode
  model.eval()
  # evaluation code for test data :
  total=0
  correct=0
  with torch.no_grad():
    for batch_features,batch_labels in test_loader:
      # Flatten the image data
      batch_features = batch_features.view(-1, 28 * 28).to(device) # Move data to device
      batch_labels = batch_labels.to(device) # Move labels to device

      outputs=model(batch_features)
      _,predicted=torch.max(outputs.data,1)
      total+=batch_labels.shape[0]
      correct+=(predicted==batch_labels).sum().item()
    accuracy=correct/total*100

  return accuracy

# CREATING STUDY USING OPTUNA

In [29]:
! pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 30.2 MB/s eta 0:00:00


In [30]:
import optuna

study = optuna.create_study(direction='maximize')

[I 2025-09-12 04:24:31,687] A new study created in memory with name: no-name-d1fc4286-e073-4bce-b8a4-268e19cdbc5b


In [31]:
study.optimize(objective,n_trials=10)

[I 2025-09-12 04:26:13,288] Trial 0 finished with value: 95.59 and parameters: {'num_hidden_layer': 2, 'neuron_per_layer': 40, 'epochs': 20, 'learning_rate': 0.0030967003391325536, 'dropout_rate': 0.4, 'batch_size': 32, 'optimizer': 'Adam', 'weight_decay': 0.00013551565383828953}. Best is trial 0 with value: 95.59.
[I 2025-09-12 04:27:14,076] Trial 1 finished with value: 96.46000000000001 and parameters: {'num_hidden_layer': 1, 'neuron_per_layer': 48, 'epochs': 30, 'learning_rate': 0.00014314655528781272, 'dropout_rate': 0.30000000000000004, 'batch_size': 64, 'optimizer': 'Adam', 'weight_decay': 1.5526910461359215e-05}. Best is trial 1 with value: 96.46000000000001.
[I 2025-09-12 04:28:48,548] Trial 2 finished with value: 97.6 and parameters: {'num_hidden_layer': 4, 'neuron_per_layer': 72, 'epochs': 50, 'learning_rate': 2.4799410454874547e-05, 'dropout_rate': 0.1, 'batch_size': 128, 'optimizer': 'Adam', 'weight_decay': 0.0005133146513986134}. Best is trial 2 with value: 97.6.
[I 2025-0

In [32]:
print(study.best_value)
print(study.best_params)

97.6
{'num_hidden_layer': 4, 'neuron_per_layer': 72, 'epochs': 50, 'learning_rate': 2.4799410454874547e-05, 'dropout_rate': 0.1, 'batch_size': 128, 'optimizer': 'Adam', 'weight_decay': 0.0005133146513986134}


# IMPLEMENTING MODEL

In [33]:
# Get the best parameters from the study
best_params = study.best_params

# Extract the best parameters for the model
num_hidden_layer = best_params['num_hidden_layer']
neuron_per_layer = best_params['neuron_per_layer']
dropout_rate = best_params['dropout_rate']

# Define input and output dimensions
input_dim = 784 # For flattened 28x28 MNIST images
output_dim = 10 # For 10 digits (0-9)

# Create the model with the best parameters
model = MyNN(input_dim, output_dim, num_hidden_layer, neuron_per_layer, dropout_rate)

# Move the model to the appropriate device (GPU if available)
model.to(device)

print("Model created with best hyperparameters:")
print(model)

Model created with best hyperparameters:
MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=72, bias=True)
    (1): BatchNorm1d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Linear(in_features=72, out_features=72, bias=True)
    (5): BatchNorm1d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.1, inplace=False)
    (8): Linear(in_features=72, out_features=72, bias=True)
    (9): BatchNorm1d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.1, inplace=False)
    (12): Linear(in_features=72, out_features=72, bias=True)
    (13): BatchNorm1d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.1, inplace=False)
    (16): Linear(in_features=72, out_features=10, bias=True)
  )
)


In [35]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()

# Get optimizer from best parameters
optimizer_name = best_params['optimizer']
learning_rate = best_params['learning_rate']
weight_decay = best_params['weight_decay']

if optimizer_name == 'Adam':
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
elif optimizer_name == 'SGD':
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
else: # RMSprop
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Get the number of epochs and batch size from best parameters
epochs = best_params['epochs']
batch_size = best_params['batch_size']

# Re-create DataLoaders with the best batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# Training loop
print("Starting training...")
for epoch in range(epochs):
    model.train() # Set the model to training mode
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # Flatten the input images
        inputs = inputs.view(inputs.size(0), -1)

        # Move inputs and labels to the device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print epoch loss
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

print("Training finished.")

Starting training...
Epoch 1/50, Loss: 1.9102
Epoch 2/50, Loss: 1.2983
Epoch 3/50, Loss: 0.9607
Epoch 4/50, Loss: 0.7453
Epoch 5/50, Loss: 0.6048
Epoch 6/50, Loss: 0.5050
Epoch 7/50, Loss: 0.4371
Epoch 8/50, Loss: 0.3831
Epoch 9/50, Loss: 0.3427
Epoch 10/50, Loss: 0.3141
Epoch 11/50, Loss: 0.2884
Epoch 12/50, Loss: 0.2672
Epoch 13/50, Loss: 0.2518
Epoch 14/50, Loss: 0.2343
Epoch 15/50, Loss: 0.2293
Epoch 16/50, Loss: 0.2152
Epoch 17/50, Loss: 0.2054
Epoch 18/50, Loss: 0.1952
Epoch 19/50, Loss: 0.1916
Epoch 20/50, Loss: 0.1839
Epoch 21/50, Loss: 0.1788
Epoch 22/50, Loss: 0.1744
Epoch 23/50, Loss: 0.1700
Epoch 24/50, Loss: 0.1648
Epoch 25/50, Loss: 0.1610
Epoch 26/50, Loss: 0.1587
Epoch 27/50, Loss: 0.1542
Epoch 28/50, Loss: 0.1509
Epoch 29/50, Loss: 0.1463
Epoch 30/50, Loss: 0.1448
Epoch 31/50, Loss: 0.1409
Epoch 32/50, Loss: 0.1381
Epoch 33/50, Loss: 0.1378
Epoch 34/50, Loss: 0.1367
Epoch 35/50, Loss: 0.1313
Epoch 36/50, Loss: 0.1304
Epoch 37/50, Loss: 0.1271
Epoch 38/50, Loss: 0.1256


# EVALUATION

In [37]:
#evaluation
#set model to eval mode
model.eval()
# evaluation code for test data :
total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in test_loader:
    # Flatten the image data
    batch_features = batch_features.view(batch_features.size(0), -1)
    #move data to gpu
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    outputs=model(batch_features)
    _,predicted=torch.max(outputs.data,1)
    total+=batch_labels.shape[0]
    correct+=(predicted==batch_labels).sum().item()
  accuracy=correct/total*100

print(f"Accuracy on test set: {accuracy:.2f}%")

Accuracy on test set: 97.45%


In [39]:
#evaluation
#set model to eval mode
model.eval()
# evaluation code for train data :
total=0
correct=0
with torch.no_grad():
  for batch_features,batch_labels in train_loader:
    # Flatten the image data
    batch_features = batch_features.view(batch_features.size(0), -1)
    #move data to gpu
    batch_features = batch_features.to(device)
    batch_labels = batch_labels.to(device)

    outputs=model(batch_features)
    _,predicted=torch.max(outputs.data,1)
    total+=batch_labels.shape[0]
    correct+=(predicted==batch_labels).sum().item()
  accuracy=correct/total*100

print(f"Accuracy on train set: {accuracy:.2f}%")

Accuracy on train set: 98.73%
